In [42]:
from datetime import datetime as dt
import itertools
import numpy as np
import os
import pickle
import torch
from torch import nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader

from lstm import LSTM
from music_dataloader import create_split_loaders
from torch_utils import setup_device

from train_eval_utils import *

In [50]:
def sample_lstm(dictionary, sample_len = 100, model_path='models/lstm.pt', start_char = '<start>', temperature = 1, num_nodes = 100):
    
    out_list = []
    # Load Model
    model = LSTM(len(dictionary), num_nodes, len(dictionary), computing_device, 1)
    model.load_state_dict(torch.load(model_path))
    model.eval()
    model.reset_state()
    
    #Init Starting Sample
    current_sequence_input = torch.zeros(1, 1, len(dictionary))
    current_sequence_input [0,0,dictionary.class_to_index[start_char]]=1
    current_sequence_input = current_sequence_input.to(computing_device)
    
    out_list.append(start_char)
    
    for i in range(sample_len):
        
        output = model(current_sequence_input)
        probabilities = nn.functional.softmax(output.div(temperature), dim=2)
        prob = probabilities.data.cpu().numpy().flatten().astype('float64')
        prob = prob / prob.sum()
        out_ind = np.argmax(np.random.multinomial(1, prob, size=1)[0])
#         out_list.append(dictionary.index_to_class[torch.argmax(probabilities.data)])
        out_list.append(dictionary.index_to_class[out_ind])

        current_sequence_input = torch.zeros(1, 1, len(dictionary))
        current_sequence_input [0,0,dictionary.class_to_index[start_char]]=1
        current_sequence_input = current_sequence_input.to(computing_device)
    
    return out_list
    

In [84]:
num_nodes = 50
model_path = 'training_logs/models_50/50nodes.pt'
temp = 1
sample_len = 100

In [85]:
seq_length = 100

computing_device = setup_device()
train_loader, val_loader, test_loader, dictionary = create_split_loaders(seq_length)


CUDA is supported


In [86]:
samp = sample_lstm(dictionary,sample_len, model_path, '<start>', temp ,num_nodes)
print("".join(samp))

<start>



@v

@
@@@@@@@@@@@@@@
@
@@@@@@@@@@


@
@@@@@@@@@@@
@
@@@@@@@@@@

@@
@@@@@@@@@@@
@@@@@@@@@@@@@@@
@
